# 1. start with small idea of RAG - search by keywords matching

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import functools
from concurrent.futures import ThreadPoolExecutor
import time

c:\Users\anaconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

# Configuration

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [3]:
# Performance improvement: Use caching for context loading
@functools.lru_cache(maxsize=None)
def load_context():
    """Load and cache context data to avoid repeated file I/O"""
    context = {}
    
    # Load employees with threading for faster I/O
    def load_employee_files():
        employee_context = {}
        employees = glob.glob("knowledge-base/employees/*")
        
        def load_single_employee(employee):
            name = employee.split(' ')[-1][:-3]
            try:
                with open(employee, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {employee}: {e}")
                return name, ""
        
        # Use ThreadPoolExecutor for concurrent file loading
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_employee, employees)
            employee_context.update(dict(results))
        
        return employee_context
    
    # Load schools with threading
    def load_school_files():
        school_context = {}
        schools = glob.glob("knowledge-base/schools/*")
        
        def load_single_school(school):
            name = school.split(os.sep)[-1][:-3]
            try:
                with open(school, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {school}: {e}")
                return name, ""
        
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_school, schools)
            school_context.update(dict(results))
        
        return school_context
    
    # Load both types concurrently
    with ThreadPoolExecutor(max_workers=2) as executor:
        employee_future = executor.submit(load_employee_files)
        school_future = executor.submit(load_school_files)
        
        context.update(employee_future.result())
        context.update(school_future.result())
    
    return context

In [4]:
# Load context once at startup
context = load_context()
print(f"Loaded context with {len(context)} documents: {list(context.keys())}")

Loaded context with 11 documents: ['Kim', 'Lee', 'Lan', 'Jae', 'Park', 'Ewha Womans University', 'KAIST', 'Korea University', 'POSTECH', 'Seoul National University', 'Yonsei University']


In [5]:
context["Lan"]

'# Bà Nguyễn Thị Lan - Giám Đốc Điều Hành\n\n## Thông Tin Cá Nhân\n\n**Họ và tên**: Nguyễn Thị Lan\n**Chức vụ**: Giám Đốc Điều Hành (CEO)  \n**Tuổi**: 42  \n**Quốc tịch**: Việt Nam  \n**Ngôn ngữ**: Tiếng Việt (bản ngữ), Tiếng Hàn (Thành thạo - TOPIK 6), Tiếng Anh (Thành thạo - IELTS 8.0)  \n\n## Liên Hệ\n\n📞 **Điện thoại**: 0901-111-222  \n📧 **Email**: lan.nguyen@koreastudyvn.com  \n📧 **Email cá nhân**: ceo@koreastudyvn.com  \n🏢 **Văn phòng**: Tầng 12, Lotte Center, Hà Nội  \n\n## Học Vấn\n\n### Bằng Cấp Chính\n- **2010**: Thạc sĩ Quản trị Kinh doanh (MBA) - Đại học Yonsei, Seoul, Hàn Quốc\n- **2005**: Cử nhân Quan hệ Quốc tế - Đại học Ngoại thương, Hà Nội, Việt Nam\n\n### Chứng Chỉ Bổ Sung\n- **2018**: Chứng chỉ Tư vấn Giáo dục Quốc tế - ICEF (International Consultants for Education and Fairs)\n- **2016**: Chứng chỉ Quản lý Dự án - PMP (Project Management Professional)\n- **2012**: Chứng chỉ Tiếng Hàn TOPIK Level 6\n\n## Kinh Nghiệm Làm Việc\n\n### 2018 - Hiện tại: Giám Đốc Điều Hành\

In [6]:
system_message = (
    "Bạn là một chuyên gia tư vấn du học Hàn Quốc tại trung tâm Korea Study. "
    "Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến trung tâm, nhân viên, trường học, và thông tin visa một cách ngắn gọn và chính xác. "
    "Nếu bạn không biết câu trả lời, hãy nói rõ rằng bạn không biết. "
    "Tuyệt đối không bịa ra thông tin nếu không có ngữ cảnh liên quan được cung cấp."
)

In [7]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context  

In [8]:
get_relevant_context("Who is Timi?")

[]

In [9]:
get_relevant_context("who is Nguyễn Thị Lan")

['# Bà Nguyễn Thị Lan - Giám Đốc Điều Hành\n\n## Thông Tin Cá Nhân\n\n**Họ và tên**: Nguyễn Thị Lan\n**Chức vụ**: Giám Đốc Điều Hành (CEO)  \n**Tuổi**: 42  \n**Quốc tịch**: Việt Nam  \n**Ngôn ngữ**: Tiếng Việt (bản ngữ), Tiếng Hàn (Thành thạo - TOPIK 6), Tiếng Anh (Thành thạo - IELTS 8.0)  \n\n## Liên Hệ\n\n📞 **Điện thoại**: 0901-111-222  \n📧 **Email**: lan.nguyen@koreastudyvn.com  \n📧 **Email cá nhân**: ceo@koreastudyvn.com  \n🏢 **Văn phòng**: Tầng 12, Lotte Center, Hà Nội  \n\n## Học Vấn\n\n### Bằng Cấp Chính\n- **2010**: Thạc sĩ Quản trị Kinh doanh (MBA) - Đại học Yonsei, Seoul, Hàn Quốc\n- **2005**: Cử nhân Quan hệ Quốc tế - Đại học Ngoại thương, Hà Nội, Việt Nam\n\n### Chứng Chỉ Bổ Sung\n- **2018**: Chứng chỉ Tư vấn Giáo dục Quốc tế - ICEF (International Consultants for Education and Fairs)\n- **2016**: Chứng chỉ Quản lý Dự án - PMP (Project Management Professional)\n- **2012**: Chứng chỉ Tiếng Hàn TOPIK Level 6\n\n## Kinh Nghiệm Làm Việc\n\n### 2018 - Hiện tại: Giám Đốc Điều Hành

In [10]:
get_relevant_context("who is Nguyễn Thị Lan and introduce about Yonsei University")

['# Bà Nguyễn Thị Lan - Giám Đốc Điều Hành\n\n## Thông Tin Cá Nhân\n\n**Họ và tên**: Nguyễn Thị Lan\n**Chức vụ**: Giám Đốc Điều Hành (CEO)  \n**Tuổi**: 42  \n**Quốc tịch**: Việt Nam  \n**Ngôn ngữ**: Tiếng Việt (bản ngữ), Tiếng Hàn (Thành thạo - TOPIK 6), Tiếng Anh (Thành thạo - IELTS 8.0)  \n\n## Liên Hệ\n\n📞 **Điện thoại**: 0901-111-222  \n📧 **Email**: lan.nguyen@koreastudyvn.com  \n📧 **Email cá nhân**: ceo@koreastudyvn.com  \n🏢 **Văn phòng**: Tầng 12, Lotte Center, Hà Nội  \n\n## Học Vấn\n\n### Bằng Cấp Chính\n- **2010**: Thạc sĩ Quản trị Kinh doanh (MBA) - Đại học Yonsei, Seoul, Hàn Quốc\n- **2005**: Cử nhân Quan hệ Quốc tế - Đại học Ngoại thương, Hà Nội, Việt Nam\n\n### Chứng Chỉ Bổ Sung\n- **2018**: Chứng chỉ Tư vấn Giáo dục Quốc tế - ICEF (International Consultants for Education and Fairs)\n- **2016**: Chứng chỉ Quản lý Dự án - PMP (Project Management Professional)\n- **2012**: Chứng chỉ Tiếng Hàn TOPIK Level 6\n\n## Kinh Nghiệm Làm Việc\n\n### 2018 - Hiện tại: Giám Đốc Điều Hành

In [11]:
get_relevant_context("who is nguyen thi la and introduce about yonsei")

[]

In [12]:
def add_context(message):
    """Add relevant context to message"""
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nNhững thông tin sau có thể hữu ích cho việc trả lời câu hỏi này:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [13]:
def chat(message, history):
    """Optimized chat function with better error handling"""
    try:
        messages = [{"role": "system", "content": system_message}] + history
        message = add_context(message)
        messages.append({"role": "user", "content": message})

        stream = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            stream=True,
            max_tokens=1000,  # Limit response length for faster generation
            temperature=0.7
        )

        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                yield response
    except Exception as e:
        yield f"Xin lỗi, đã có lỗi xảy ra: {str(e)}"

In [14]:
# Launch first version
print("Launching keyword-based RAG chatbot...")
view = gr.ChatInterface(chat, type="messages").launch()

Launching keyword-based RAG chatbot...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# 2. RAG bigger idea with vector search - optimized version

In [15]:
# imports for langchain

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [16]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

#text_loader_kwargs = {'encoding': 'utf-8'}
# Nếu dòng trên không hoạt động, người dùng Windows có thể dùng dòng dưới thay thế
text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print("Total documents loaded:", len(documents))


Total documents loaded: 17


In [17]:
documents[0]

Document(metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}, page_content='# Về Công Ty - Korea Study Consultant Center\n\n## Lịch Sử Thành Lập\n\nKorea Study Consultant Center được thành lập vào năm 2018 với sứ mệnh kết nối các bạn trẻ Việt Nam với hệ thống giáo dục chất lượng cao của Hàn Quốc. Được sáng lập bởi các chuyên gia giáo dục có nhiều năm kinh nghiệm tại Hàn Quốc, chúng tôi đã hỗ trợ hơn 2,000 học sinh Việt Nam thực hiện ước mơ du học tại xứ sở kim chi.\n\n## Tầm Nhìn & Sứ Mệnh\n\n### Tầm Nhìn\nTrở thành trung tâm tư vấn du học Hàn Quốc hàng đầu tại Việt Nam, mang đến cơ hội giáo dục tốt nhất cho thế hệ trẻ Việt Nam.\n\n### Sứ Mệnh\n- Cung cấp dịch vụ tư vấn du học chuyên nghiệp và uy tín\n- Hỗ trợ toàn diện từ khâu chuẩn bị hồ sơ đến khi định cư tại Hàn Quốc\n- Xây dựng cầu nối văn hóa và giáo dục giữa Việt Nam và Hàn Quốc\n- Đảm bảo tỷ lệ thành công cao nhất cho học sinh\n\n## Giá Trị Cốt Lõi\n\n### 1. Chuyên Nghiệp\n- Đội ngũ tư vấn viên có bằ

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Slightly smaller chunks for better retrieval
    chunk_overlap=100,  # Reduced overlap for performance
    separators=["\n\n", "\n", ". ", " ", ""]  # Better separation
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

Created 155 chunks


In [38]:
chunks[9]

Document(metadata={'source': 'knowledge-base\\company\\overview.md', 'doc_type': 'company'}, page_content='## Trường Đối Tác Chính\n\n### Top Universities\n- Seoul National University (서울대학교)\n- Yonsei University (연세대학교)  \n- Korea University (고려대학교)\n- KAIST (한국과학기술원)\n- POSTECH (포항공과대학교)\n\n### Private Universities\n- Hanyang University (한양대학교)\n- Kyung Hee University (경희대학교)\n- Ewha Womans University (이화여자대학교)\n- Sogang University (서강대학교)\n- Sungkyunkwan University (성균관대학교)\n\n### Language Institutes\n- Yonsei Korean Language Institute\n- Seoul National University LEI\n- Sogang Korean Language Program\n- Ewha Language Center\n\n## Khu Vực Phục Vụ\n\n### Việt Nam\n- **Miền Bắc**: Hà Nội, Hải Phòng, Nam Định, Thái Bình\n- **Miền Trung**: Đà Nẵng, Huế, Vinh, Quy Nhon\n- **Miền Nam**: TP.HCM, Cần Thơ, Vũng Tàu, Đồng Nai')

In [21]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_types)}")

Các loại tài liệu đã tìm thấy: company, visas, schools, employees


In [22]:
for chunk in chunks:
    if 'Nguyễn Thị Lan' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='### 4. Hiệu Quả
- Rút ngắn thời gian xử lý hồ sơ
- Chi phí hợp lý, minh bạch
- Cam kết hoàn tiền nếu không đậu visa

## Thành Tựu Nổi Bật

### Số Liệu Thống Kê
- **2,000+** học sinh được hỗ trợ thành công
- **98.5%** tỷ lệ đậu visa du học
- **50+** trường đại học đối tác
- **95%** học sinh hài lòng với dịch vụ

### Giải Thưởng
- **2022**: Trung tâm tư vấn du học uy tín nhất Việt Nam
- **2021**: Đại lý chính thức xuất sắc của Đại học Yonsei
- **2020**: Top 3 công ty tư vấn du học Hàn Quốc tại Việt Nam

## Đội Ngũ Lãnh Đạo

### Giám Đốc Điều Hành
**Bà Nguyễn Thị Lan** - 15 năm kinh nghiệm trong lĩnh vực giáo dục quốc tế, từng học tập và làm việc tại Hàn Quốc 8 năm.' metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}
_________
page_content='### Thành Viên Của
- **Hiệp hội Tư vấn Du học Việt Nam (VIECA)**
- **Liên minh Giáo dục Việt Nam - Hàn Quốc**
- **Mạng lưới Đối tác Giáo dục Quốc tế (IEPN)**

### Đối Tác Chính Thức
- **Bộ Giáo dục Hàn Quốc (M

In [23]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [24]:
# Đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ HuggingFace (thay vì OpenAI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [25]:
# ta sẽ sử dụng mô hình chi phí thấp
MODEL = "gpt-4o-mini"

# Đặt tên cho database vector (có thể tùy chọn)
db_name = "vector_db"

# Kiểm tra nếu database Chroma đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [26]:
# Tạo vector store bằng Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,              # Danh sách các đoạn văn bản đã chia nhỏ
    embedding=embeddings,          # Hàm embedding (ví dụ: OpenAI hoặc HuggingFace)
    persist_directory=db_name      # Thư mục lưu trữ database
)
# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


Vectorstore created with 155 documents


In [27]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [28]:
sample_embedding

array([-0.00046667,  0.00158895,  0.01983302, ..., -0.00275115,
       -0.01760473,  0.00132589])

In [29]:
# Lấy toàn bộ vector, tài liệu và metadata từ collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Đưa embedding vào mảng numpy
vectors = np.array(result['embeddings'])

# Lưu lại văn bản
documents = result['documents']

# Trích loại tài liệu từ metadata (giả sử có 'doc_type')
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]

# Gán màu sắc tùy theo loại tài liệu
colors = [['blue', 'green', 'red', 'orange'][['company', 'employees', 'visas', 'schools'].index(t)] for t in doc_types]


In [30]:
# 2D dimension!
# Giảm số chiều của vector xuống 2D bằng t-SNE
# (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 2D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="browser")

In [31]:
from langchain.memory import ConversationBufferWindowMemory  
from langchain.chains import ConversationalRetrievalChain

In [32]:
# Tạo mô hình Chat với OpenAI
llm = ChatOpenAI(
    temperature=0.7, 
    model_name=MODEL,
)

# Thiết lập bộ nhớ hội thoại
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# Tạo retriever từ vector store (Chroma)
retriever = vectorstore.as_retriever()

# Kết nối tất cả thành một chuỗi hội thoại có khả năng truy xuất (RAG pipeline)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


C:\Users\pc hp\AppData\Local\Temp\ipykernel_13744\2637877394.py:8: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [33]:
# Test query with performance monitoring
def test_query_performance():
    """Test query with timing"""
    query = "Bạn có thể mô tả ngắn gọn về Korea study center không?"
    start_time = time.time()
    result = conversation_chain.invoke({"question": query})
    end_time = time.time()
    
    print(f"Query processed in {end_time - start_time:.2f} seconds")
    print("Answer:", result["answer"])
    if "source_documents" in result:
        print(f"Used {len(result['source_documents'])} source documents")

In [34]:
test_query_performance()

Query processed in 7.58 seconds
Answer: Korea Study Consultant Center (KSCC) là một công ty tư vấn giáo dục quốc tế được thành lập vào năm 2018, với sứ mệnh kết nối học sinh Việt Nam với hệ thống giáo dục chất lượng cao của Hàn Quốc. Công ty đã hỗ trợ hơn 2,000 học sinh du học tại Hàn Quốc, chuyên tư vấn các chương trình thạc sĩ và tiến sĩ, đặc biệt trong các ngành STEM. KSCC có mối quan hệ đối tác với 68 trường đại học và cam kết đảm bảo chất lượng tư vấn đạt tiêu chuẩn quốc tế. Trụ sở chính đặt tại Hà Nội và có chi nhánh tại TP.HCM.


In [35]:
# set up a new conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [36]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [37]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [39]:
# Hãy cùng tìm hiểu xem điều gì được gửi phía sau hậu trường

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)

query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
**Q: Job prospects sau tốt nghiệp như thế nào?**
A: Rất tốt, đặc biệt cho R&D positions. Many graduates become leaders trong tech industry.

### Khoa Y học
- Bác sĩ đa khoa (6 năm)
- Nha khoa
- Y học dự phòng

### Khoa Kinh doanh
- Quản trị kinh doanh
- Kinh tế học
- Tài chính ngân hàng

### Khoa Nhân văn
- Ngôn ngữ và văn học Hàn Quốc
- Lịch sử Hàn Quốc
- Triết học phương Đông

## Yêu cầu tuyển sinh

### Sinh viên quốc tế (Undergraduate)
- **Học lực:** Tốt nghiệp THPT với điểm trung bình tối thiểu 3.5/4.0
- **Tiếng Anh:** TOEFL iBT 88+ hoặc IELTS 6.5+
- **Tiếng Hàn:** TOPIK Level 4+ (khuyến khích Level 5-6)
- **Hồ sơ:** Bằng tốt nghiệp, bảng điể

In [63]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)

query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
**Q: Job prospects sau tốt nghiệp như thế nào?**
A: Rất tốt, đặc biệt cho R&D positions. Many graduates become leaders trong tech industry.

### Khoa Y học
- Bác sĩ đa khoa (6 năm)
- Nha khoa
- Y học dự phòng

### Khoa Kinh doanh
- Quản trị kinh doanh
- Kinh tế học
- Tài chính ngân hàng

### Khoa Nhân văn
- Ngôn ngữ và văn học Hàn Quốc
- Lịch sử Hàn Quốc
- Triết học phương Đông

## Yêu cầu tuyển sinh

### Sinh viên quốc tế (Undergraduate)
- **Học lực:** Tốt nghiệp THPT với điểm trung bình tối thiểu 3.5/4.0
- **Tiếng Anh:** TOEFL iBT 88+ hoặc IELTS 6.5+
- **Tiếng Hàn:** TOPIK Level 4+ (khuyến khích Level 5-6)
- **Hồ sơ:** Bằng tốt nghiệp, bảng điể

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]


In [ ]:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?
Assistant: Trong công ty, Sarah Park (Nguyễn Sarah) đã tốt nghiệp từ Đại học Ngoại thương Hà Nội.
Follow Up Input: Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
**Q: Job prospects sau tốt nghiệp như thế nào?**
A: Rất tốt, đặc biệt cho R&D positions. Many graduates become leaders trong tech industry.

#

# 3 Improve RAG Ensemble Hybrid Retrieval.

In [66]:
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever
from typing import List, Dict

# 1. Fix keyword retriever to search content, not just title
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def get_relevant_documents(self, query: str) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(Document(page_content=content, metadata={"source": title}))
        return relevant_docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

# 2. Use keyword + properly configured vector retriever
keyword_retriever = KeywordRetriever(context_dict=context)
vector_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 30}
)

# 3. Ensemble
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 4. Conversation chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)


C:\Users\pc hp\AppData\Local\Temp\ipykernel_8448\2883139772.py:6: DeprecationWarning:

Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`

C:\Users\pc hp\AppData\Local\Temp\ipykernel_8448\2883139772.py:6: DeprecationWarning:

Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`



In [88]:
query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: nhân viên nào tốt nghiệp đại học ngoại thương
Assistant: Trong thông tin đã cung cấp, có một nhân viên tốt nghiệp từ Đại học Ngoại thương Hà Nội, đó là Park Sarah (Nguyễn Sarah), với bằng cử nhân Quan hệ Quốc tế.
Human: Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?
Assistant: Có thông tin về Bà Nguyễn Thị Lan, Giám Đốc Điều Hành (CEO) của Korea Study Consultant Center, đã tốt nghiệp Cử nhân Quan hệ Quốc tế từ Đại học Ngoại thương, Hà Nội, Việt Nam.
Follow Up Input: Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the fo

In [40]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)